In [43]:
import xml.etree.ElementTree as ET
import re
from nltk.corpus import stopwords
import pandas as pd

In [44]:
# read the latest mass production table
# for 6.05 clean
# path_new = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.txt"
# fin = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
# fout_folder = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# for 7.17 clean
path_new = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new_cleaned.txt"
fin = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new_cleaned.brd"
fout_folder = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/FinalBRDs/"

# for 6.05 translate
# path_new = "./Output_translated_folder/6.05 HTML/6.05 HTML/MassProduction/6.05_translated_no_mark.txt"
# fin = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
# fout_folder = "./Output_translated_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# for 7.17 translate
# path_new = "./Output_translated_folder/7.17 HTML/7.17 HTML/MassProduction/7.17_translated_no_mark.txt"
# fin = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new_cleaned.brd"
# fout_folder = "./Output_translated_folder/7.17 HTML/7.17 HTML/FinalBRDs/"

In [45]:
table_new = pd.read_csv(path_new, sep="\t", index_col=0, keep_default_na=False)

In [46]:
# replace variable with value in the latest mass production table
# iterate column and row in table_new
for column in range(table_new.shape[1]):
    for row in range(table_new.shape[0]):
        content_new = str(table_new.iloc[row, column])
        start_pattern = "%\("
        # count the number of the replacement in one variable(content_new)
        count = [match.start() for match in re.finditer(start_pattern, str(content_new))]
        for _ in range(len(count)):
            # find the variable
            start_pattern = "%\("
            start_index = [match.start() for match in re.finditer(start_pattern, str(content_new))]
            end_pattern = "\)%"
            end_index = [match.start() for match in re.finditer(end_pattern, str(content_new))]
            # no need for replacement
            if start_index == []:
                continue
            else:
                variable = content_new[start_index[0]: end_index[0]+2]
                # find the corresponding column name, and then find the value
                column_name = table_new.columns[column]
                try:
                    value = table_new.loc[variable, column_name] 
                    # print(variable, value)
                    content_new = content_new.replace(variable, value)
                    table_new.iloc[row, column] = content_new
                except:
                    print(variable + " doesn't exist")
table_new.head()

,IntroProblem,Problem1,Problem2,Problem3,Problem5,Problem6,Problem7,Problem8,Problem9,Problem10,...,Problem15,Problem16,Problem17,Problem18,Problem19,Problem20,Problem21,Problem22,Problem23,Problem24
Problem Name,,,,,,,,,,,,,,,,,,,,,
%(startStateNodeName)%,state116,state56,state27,state2,state116,state116,state116,state116,state116,state116,...,state116,state116,state116,state116,state116,state116,state116,state116,state116,state116
%(ProblemStatement)%,Kelly has 50 senior pictures. She is giving 32...,"As part of their Get Fit campaign, the city of...",Timothy is flying from Los Angeles to New York...,Ed is installing carpeting in his house. He ha...,Albert is reading a book that is 200 pages lon...,A radio program that consists of 3 fictional s...,Jane is returning a poorly developed photograp...,A political science student at a city universi...,One local newscast tries to lure viewers with ...,At the 2002 Winter Olympic Games in Salt Lake ...,...,Joan is shocked when she received her very fir...,A president's approval rating is determined by...,An encouraging survey by a local health organi...,The cafeteria staff of your school thinks it h...,Lisa is purchasing a pair of jeans from the cl...,You decide to buy a new computer on one of the...,"Julie just took a job as a telemarketer, selli...",Chelsea completed 240 math problems in a compe...,Alex practiced his driving regularly in the we...,"A dinner costs $35.00, and the waiter receives..."
%(Part)%,32,16590,775,243,72,15,0.12,36,1625,10,...,710,1300000,1980,10,48.45,105,189,30,168,5.25
%(Part-Hint1)%,How many pictures did Kelly give to her family?,What is the number of people who took part in ...,How many miles has Timothy already covered?,How much carpeting has Ed already installed in...,How many pages did Albert already read from th...,What is the length of one story?,How much credit did the shop return to Jane?,How many people answered the question correctly?,How many viewers have won the contest in the l...,How many athletes from team USA won gold medal...,...,How much of Joan's salary went to taxes?,How many citizens approved of the president's ...,How many adults regularly participated in phys...,How many students tried the healthy meal?,How much did Lisa spend on the pair of jeans?,How much did you save by not paying tax?,How many customers agreed to buy the company's...,How many geometry problems were there in the m...,What was Alex's test score?,How much tip did the waiter receive?
%(Alex-ex-hint)%,,,,,,,,,,,...,,,,,,,,,If 7 points are deducted from the 175 possible...,


In [47]:
for i in range(len(table_new.columns)):
    column_name = table_new.columns[i]
    # print(table_new.columns[i])
    fout = fout_folder + str(table_new.columns[i]) + ".brd"
    # print(fout)
    count_line = 0
    count_text = 0
    with open(fin, 'r') as infile, open(fout, 'w+') as outfile:
        for line in infile:
            line = line.replace('\r', '')
            line_str = str(line)
            # replace massproduce
            start_pattern_problem_name = "<ProblemName>"
            start_index_problem_name = [match.start() for match in re.finditer(start_pattern_problem_name, line_str)]
            end_pattern_problem_name = "</ProblemName>"
            end_index_problem_name = [match.start() for match in re.finditer(end_pattern_problem_name, line_str)]
            start_index_problem_name, end_index_problem_name
            try:
                problem_name_old = line_str[start_index_problem_name[0]+13: end_index_problem_name[0]]
                line_str = line_str.replace(problem_name_old, column_name)
            except:
                pass
            # replace text in first node
            if count_text == 0:
                start_pattern_first_node = '<text>'
                start_index_first_node = [match.start() for match in re.finditer(start_pattern_first_node, line_str)]
                end_pattern_first_node = '</text>'
                end_index_first_node = [match.start() for match in re.finditer(end_pattern_first_node, line_str)]
                if start_index_first_node != []:
                    # print(start_index_first_node, count_line)
                    count_text += 1
                    # print(line_str[start_index_first_node[0]+6: end_index_first_node[0]])
                    text_old = line_str[start_index_first_node[0]+6: end_index_first_node[0]]
                    line_str = line_str.replace(text_old, column_name)
            # count the number of the replacement in one variable(line_str)
            start_pattern = "%\("
            count = [match.start() for match in re.finditer(start_pattern, str(line_str))]
            if count == []:
                line_str = line_str
            else:
                for _ in range(len(count)):
                    start_pattern = "%\("
                    start_index = [match.start() for match in re.finditer(start_pattern, str(line_str))]
                    end_pattern = "\)%"
                    end_index = [match.start() for match in re.finditer(end_pattern, str(line_str))]
                    if start_index == []:
                        line_str = line_str
                    else:
                        variable = line_str[start_index[0]: end_index[0]+2]
                    try:
                        value = table_new.loc[variable, column_name] 
                        # print(variable, value)
                        line_str = line_str.replace(variable, value)
                    except:
                        print(variable + " doesn't exist")
            count_line += 1
            # print(str(count_line) + line_str)
            outfile.write(line_str)
        print(fout.split("/")[-1] + " finished")

IntroProblem.brd finished
Problem1.brd finished
Problem2.brd finished
Problem3.brd finished
Problem5.brd finished
Problem6.brd finished
Problem7.brd finished
Problem8.brd finished
Problem9.brd finished
Problem10.brd finished
Problem11.brd finished
Problem12.brd finished
Problem13.brd finished
Problem14.brd finished
Problem15.brd finished
Problem16.brd finished
Problem17.brd finished
Problem18.brd finished
Problem19.brd finished
Problem20.brd finished
Problem21.brd finished
Problem22.brd finished
Problem23.brd finished
Problem24.brd finished
